In [58]:
from ciml import gather_results
from ciml import tf_trainer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx
import matplotlib.colors as pltcolors
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import collections
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from subunit2sql.db import api
from tensorflow.python.training import adagrad
import tensorflow as tf

In [59]:
data_path = '/Users/andreafrittoli/ciml/cimlosseu2018labels'
dataset = 'usr-1min-status'
experiment = 'dnn-3x10-100epochs-bs128'

In [60]:
experiment_data = gather_results.load_experiment(
        experiment, data_path=data_path)
dataset_data = gather_results.load_model_config(
        dataset, data_path=data_path)

In [61]:
dataset_data

{'sample_interval': '1min',
 'num_features': 92,
 'normalization_params': {'usr36': [47.78779535299466, 66.78571666666669],
  'usr45': [34.97598699961236, 67.54776666666663],
  'usr71': [16.412039195091456, 61.55],
  'usr40': [42.62256502380952, 68.99493333333332],
  'usr89': [3.098859883400581, 39.92565],
  'usr9': [18.75726983948443, 47.28675000000001],
  'usr47': [32.046536679667106, 66.79676666666668],
  'usr55': [24.44148179865578, 70.87521666666666],
  'usr2': [10.968404160413504, 5.9468833333333295],
  'usr91': [2.5669283840034263, 38.22546666666668],
  'usr43': [38.05329771467986, 75.56703333333336],
  'usr62': [22.122285046790466, 61.53251666666667],
  'usr7': [14.930780249582842, 34.91283333333334],
  'usr57': [23.491646099395176, 64.76988333333335],
  'usr12': [48.50031876315884, 85.72266666666664],
  'usr28': [55.39844430124655, 64.55256666666665],
  'usr88': [3.463070510623923, 40.712816666666676],
  'usr63': [21.668888431241246, 62.658283333333344],
  'usr32': [51.9415225

In [62]:
experiment_data

{'hyper_params': {'optimizer': 'Adagrad',
  'steps': 1900,
  'learning_rate': 0.05,
  'hidden_units': [10, 10, 10],
  'epochs': '100',
  'batch_size': '128'},
 'estimator': 'tf.estimator.DNNClassifier',
 'params': {}}

experiment_data

In [63]:
estimator = experiment_data['estimator']
hyper_params = experiment_data['hyper_params']
params = experiment_data['params']
steps = int(hyper_params['steps'])
num_epochs = int(hyper_params['epochs'])
batch_size = int(hyper_params['batch_size'])
optimizer = hyper_params['optimizer']
learning_rate = float(hyper_params['learning_rate'])
class_label = dataset_data['class_label']
labels = gather_results.load_dataset(dataset, 'labels', data_path=data_path)['labels']
training_data = gather_results.load_dataset(dataset, 'training', data_path=data_path)
test_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)
label_vocabulary = None
model_dir = gather_results.get_model_folder(dataset, experiment)


In [64]:
estimator = tf_trainer.get_estimator(
        estimator, hyper_params, params, labels, model_dir,
        optimizer=adagrad.AdagradOptimizer(learning_rate=0.05),
        label_vocabulary=label_vocabulary, gpu=False)

INFO:tensorflow:Using config: {'_model_dir': '/git/github.com/mtreinish/ciml/ciml/../data/usr-1min-status/dnn-3x10-100epochs-bs128', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x3158eb390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [65]:
input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data)

In [ ]:
inputs = []
while True:
    input = input_fn()
    if not input:
        break
    else:
        inputs.append(input)

In [67]:
training_result = tf_trainer.get_training_method(estimator)(
                    input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data), steps=steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /git/github.com/mtreinish/ciml/ciml/../data/usr-1min-status/dnn-3x10-100epochs-bs128/model.ckpt-6823
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6824 into /git/github.com/mtreinish/ciml/ciml/../data/usr-1min-status/dnn-3x10-100epochs-bs128/model.ckpt.
INFO:tensorflow:loss = 0.002498201, step = 6824
INFO:tensorflow:global_step/sec: 55.2197
INFO:tensorflow:loss = 0.00093192165, step = 6924 (1.812 sec)
INFO:tensorflow:global_step/sec: 63.2253
INFO:tensorflow:loss = 0.009960286, step = 7024 (1.582 sec)
INFO:tensorflow:global_step/sec: 63.36
INFO:tensorflow:loss = 0.0004120939, step = 7124 (1.578 sec)
INFO:tensorflow:global_step/sec: 62.9212
INFO:tensorflow:loss = 0.0009846727, step = 7224 (1.590 sec)
INFO:tensorflow:glob

In [68]:
training_result.__dict__

{'_config': <tensorflow.python.estimator.run_config.RunConfig at 0x315b74748>,
 '_model_dir': '/git/github.com/mtreinish/ciml/ciml/../data/usr-1min-status/dnn-3x10-100epochs-bs128',
 '_session_config': allow_soft_placement: true,
 '_device_fn': None,
 '_model_fn': <function tensorflow.python.estimator.canned.dnn.DNNClassifier.__init__.<locals>._model_fn(features, labels, mode, config)>,
 '_params': {},
 '_warm_start_settings': None}

In [32]:
len(training_data['example_ids'])

2100

In [73]:
eval_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)

In [76]:
eval_size = len(eval_data['example_ids'])
eval_data.keys()

dict_keys(['example_ids', 'examples', 'classes'])

In [77]:
prediction = estimator.predict(input_fn=tf_trainer.get_input_fn(
                                batch_size=eval_size, num_epochs=1,
                                labels=labels, **eval_data))

In [79]:
predictions = [x for x in prediction]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /git/github.com/mtreinish/ciml/ciml/../data/usr-1min-status/dnn-3x10-100epochs-bs128/model.ckpt-8464
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [80]:
predictions

[{'logits': array([-25.077246], dtype=float32),
  'logistic': array([1.2855547e-11], dtype=float32),
  'probabilities': array([1.0000000e+00, 1.2855547e-11], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-12.841743], dtype=float32),
  'logistic': array([2.6478936e-06], dtype=float32),
  'probabilities': array([9.9999738e-01, 2.6478936e-06], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([27.046293], dtype=float32),
  'logistic': array([1.], dtype=float32),
  'probabilities': array([1.7945026e-12, 1.0000000e+00], dtype=float32),
  'class_ids': array([1]),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-27.080362], dtype=float32),
  'logistic': array([1.7343952e-12], dtype=float32),
  'probabilities': array([1.0000000e+00, 1.7343953e-12], dtype=float32),
  'class_ids': array([0]),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-28.977806], dtyp

In [87]:
p_classes = [x["class_ids"][0] for x in predictions]
actual_classes = eval_data["classes"]
classes = zip(p_classes, actual_classes)
counter = collections.Counter(classes)
sorted(counter.values(), reverse=True)[:15]

[859, 25, 11, 5]

In [88]:
counter

Counter({(0, 0): 859, (1, 1): 25, (1, 0): 5, (0, 1): 11})